# WE03
# Bapuji Satyala 
# U61600601

In this Notebook we are generating a synthetic dataset which has two features and one target where we look at the data and do preprocessing if possible and save it to a csv file and this csv file will be further used in another notebook where we  use the data set for fitting it to the  Support vector Classification(SVC) by implementing the Hyper Parameter tuning.

We are trying to predict if there will be crop yield or not by considering the Temperature and Humidity as the factors that are influencing the crop yield which is a case of classification.

# Importing the required libraries that we will be using in this note book.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
np.random.seed(1)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import fbeta_score,make_scorer
from sklearn.metrics import confusion_matrix


### Step 1: Generation of data set and storing it into data frame df 

In [2]:
df = pd.DataFrame({'TEMPERATURES_DEVIATION': np.random.randint(low=0,high=30,size=1000)})

I have generated the synthetic data of Temperature deviations where the minimum temperature deviation is 0 and the Maximum temperatire deviation of 30  by using the np.random.randint to generate random integers  and storing them in the column TEMPERATURES_DEVIATION of the data frame.


In [3]:
df['SOIL_MOISTURE'] = pd.DataFrame({'SOIL_MOISTURE': np.random.randint(low=0,high=100,size=1000).round().astype(int)})

I have generated the synthetic data of Soil Moisture Percentages  where the minimum Soil Moisture Percentages  is 0 and the Maximum temperatire deviation of 100 by using the np.random.randint to generate random integers and storing them in the column SOIL_MOISTURE of the data frame.


Now we are looking at the top 5 observations of the dataframe which contain the dat aof Soil Moisture and the temperture difference 

In [4]:
df.head(5)

,TEMPERATURES_DEVIATION,SOIL_MOISTURE
0,5,33
1,11,72
2,12,68
3,8,39
4,9,32


Here we are creating a new column CROP_YIELD based on the condition that the Values in the data frame of the column TEMPERATURES_DEVIATION is lesser than 15 and also the SOIL_MOISTURE percentage should be greater than 50 then we are considering it as yield and considering it as 1 if the condition fails we are considering as no yield and assigning 0 which is a boolean condition

In [5]:
df['CROP_YIELD'] = ((df['TEMPERATURES_DEVIATION'] <15) & (df['SOIL_MOISTURE'] > 50)).astype(int)


Now we are looking at the top 5 observations of the dataframe which contain the data of Soil Moisture and the temperture difference along with the CROP_YIELD information as well.

### Step 2: Preprocessing the Data

In [6]:
df.head()

,TEMPERATURES_DEVIATION,SOIL_MOISTURE,CROP_YIELD
0,5,33,0
1,11,72,1
2,12,68,1
3,8,39,0
4,9,32,0


We are looking at the number of rows and columns in the data frame 

In [7]:
df.shape

(1000, 3)

Checking if there are any null values in the data frame and additing them to get more clear understanding on data if there are any nulls in the data frame or not

In [8]:
df.isna().sum()

TEMPERATURES_DEVIATION    0
SOIL_MOISTURE             0
CROP_YIELD                0
dtype: int64

We are looking at the statstics of the data in the dataframe, from the below we can see the minimum values, maximum values and the mean and standard deviadtion in each column of the data frame 

In [9]:
df.describe()

,TEMPERATURES_DEVIATION,SOIL_MOISTURE,CROP_YIELD
count,1000.000000,1000.000000,1000.000000
mean,14.421000,50.655000,0.270000
std,8.767476,28.536403,0.444182
min,0.000000,0.000000,0.000000
25%,7.000000,26.000000,0.000000
50%,14.000000,52.000000,0.000000
75%,22.000000,76.000000,1.000000
max,29.000000,99.000000,1.000000


Here we are checking if the generated dataset is balanced or not where Value_cpunts() will give the unique values in the CROP_YIELD column of the dataframe. we are only concerened about the CROP_YIELD as it it will be Target Column.

In [10]:
df['CROP_YIELD'].value_counts()

0    730
1    270
Name: CROP_YIELD, dtype: int64

We can see that 730 observations are 0 which is no yield and 270 which is yield.

### Step 3: Splitting the data into Train set and the Test set where our target column is sti

We are assigning the features to X dataframe by dropping the column CROP_YIELD and assigning the target CROP_YIELD to Y 

In [11]:
X=df.drop('CROP_YIELD',axis=1)
y=df['CROP_YIELD']


The 'train_test_split'  is to divide the dataset into training and testing sets. It separates the features ('X_train' and 'X_test') and the target variable ('y_train' and 'y_test') with a 20% test size, i have considered the 80% of training set as the total number of observations are less

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)


### Step 4: Balancing the data 

Here we are balancing the dataset and we are using the RandomUnderSampler for attaining the goal, The random undersampler is used as the distribution of the classes are skewed as the data is looking biased,the randomunders sampler randomly deletes the  the majority class in the train set data

In [14]:
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
#adasyn = ADASYN()
#smote = SMOTE()

Here we are Creating a data frame named performance to store the metrices that are required to evaluate the performance and efficieny of all the Models we are going to fit. We are using the fbeta_score to calculate the f2 score, where f2 score is calculated when we want to pioritize the recall rather than the precision as the minority class is quite smaller than the majority class.

In [15]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": [], "F2": [], "Parameters": []})
f2_scorer = make_scorer(fbeta_score, beta=2)

Fitting the models

The Support Vector Machine(SVM) is more effective both in linear and non linear decision boundaries, where the svm always looks for a hyperplane that seperates the data points into different classes where the data points that are close to the hyperplane are the support vectors and the line that separates is the hyper plane.We do have three Kernels in the SVM, these kernels will help the SVM to handle the non linear classification without having to explicitly perform the data transformation which can be expensive computationally.

We are implementing the Hyper parameter tuning where we find the optimal hyperparameters as we know that the paramters can impact the performace of the model. We are using the Grid search where we define the set of parameters and the we check the performance of the model by trying these combinations.



### Step 5: Fitting the models

#### Implementing Support Vector Machine with Linear as Kernel

We are consider the below C values, where C is the factor that controls the regularization strength.

In [16]:
# defining parameter range 
param_grid = {'C': [0.001,0.01, 0.1, 0.5, 1, 5, 10, 50, 100],  
              'kernel': ['linear']}
  
#grid = GridSearchCV(SVC(), param_grid, scoring='f1', refit = True, verbose = 3, n_jobs=-1) 

grid = GridSearchCV(SVC(), param_grid, scoring=f2_scorer, refit = True, verbose = 3, n_jobs=-1) 
  
# fitting the model for grid search 
_ = grid.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


The total fits are 45 as we can see that 5 folds are being performed by testing the 9 different combinations.

In [17]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)

y_pred = grid.predict(X_test) 

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f2 = fbeta_score(y_test, y_pred, beta=2)


performance = pd.concat([performance, pd.DataFrame({"model": ["SVM Linear"], "Accuracy": [accuracy], "Precision": [precision], "Recall": [recall], "F1": [f1], "F2": [f2], "Parameters": [grid.best_params_]})])


{'C': 0.001, 'kernel': 'linear'}
SVC(C=0.001, kernel='linear')


Here we are capturng the best C value which is 0.1 for the kernel linear 

Generating The Confusion Matrix

In [18]:
CM_L= confusion_matrix(y_test,y_pred)
CM_L

array([[122,  12],
       [  9,  57]], dtype=int64)

####  Implementing Support Vector Machine with rbf as Kernel

The radial basis function which uses the Gaussian radial basis function to transform the data points into high dimensional space without computing the transformed feature and the gama value controls the widdth of the Gaussian function where when it is small the decision boundaries are much more smoother and when it is large the detailing of the data will be more. 

In [19]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}
  
grid = GridSearchCV(SVC(), param_grid, scoring=f2_scorer, refit = True, verbose = 3, n_jobs=-1) 
  
# fitting the model for grid search 
_ = grid.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


The total fits are 100 as we can see that 5 folds are being performed by testing the 20 different combinations.

In [20]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)

y_pred = grid.predict(X_test) 

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f2 = fbeta_score(y_test, y_pred, beta=2)


performance = pd.concat([performance, pd.DataFrame({"model": ["SVM rbf"], "Accuracy": [accuracy], "Precision": [precision], "Recall": [recall], "F1": [f1], "F2": [f2],"Parameters": [grid.best_params_]})])


{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, gamma=0.01)


Here we are capturing the best C value which is 100 and the Gama value is 0.01 for the kernel rbf

Generating The Confusion Matrix

In [21]:
CM_RBF= confusion_matrix(y_test,y_pred)
CM_RBF

array([[132,   2],
       [  0,  66]], dtype=int64)

###  Implementing Support Vector Machine with Poly as Kernel

The Coef0 will effect the flexibilty of the model so we are trying down a range of values

In [22]:
# defining parameter range 
param_grid = {'C': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100],  
              'coef0': [0.01, 0.1, 0.5, 1, 5, 10, 50, 100],
              'kernel': ['poly']}
  
grid = GridSearchCV(SVC(), param_grid, scoring=f2_scorer, refit = True, verbose = 3, n_jobs=-1) 
  
# fitting the model for grid search 
_ = grid.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


The total fits are 320 as we can see that 5 folds are being performed by testing the 64 different combinations.

In [23]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)

y_pred = grid.predict(X_test) 

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f2 = fbeta_score(y_test, y_pred, beta=2)



performance = pd.concat([performance, pd.DataFrame({"model": ["SVM Poly"], "Accuracy": [accuracy], "Precision": [precision], "Recall": [recall], "F1": [f1], "F2": [f2], "Parameters": [grid.best_params_]})])

{'C': 100, 'coef0': 100, 'kernel': 'poly'}
SVC(C=100, coef0=100, kernel='poly')


Here we are capturing the best C value which is 100 and the Coef value is 5 for the kernel Poly 

Generating The Confusion Matrix

In [24]:
CM_P= confusion_matrix(y_test,y_pred)
CM_P

array([[132,   2],
       [  2,  64]], dtype=int64)

We are printing the metrices of all the SVM models we have implemented 

In [25]:
performance

,model,Accuracy,Precision,Recall,F1,F2,Parameters
0,SVM Linear,0.895,0.826087,0.863636,0.844444,0.855856,"{'C': 0.001, 'kernel': 'linear'}"
0,SVM rbf,0.990,0.970588,1.000000,0.985075,0.993976,"{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}"
0,SVM Poly,0.980,0.969697,0.969697,0.969697,0.969697,"{'C': 100, 'coef0': 100, 'kernel': 'poly'}"


# Summary

I have Genereated  a synthetic dataset with two inputs and one output where my inputs describes the  TEMPERATURES_DEVIATION
 and the SOIL_MOISTURE and the output is the CROP_YIELD which is dependednt on the input variables and done the preprocessing of the data and splitted the data by train test split method.The dataset looks imbalanced so i have used the RandomUndersampler technique to balnce the data set and fitted the  Support Vector Machine model by using hyper parameter tuning to test different combinations of parameters by using Grid search abnd got the below results.
 #### ACCURACY
- The accuracy of the SVM model with Kernel as Linear is 0.895
- The accuracy of the SVM model with Kernel as rbf is 0.985
- The accuracy of the SVM model with Kernel as poly is 0.990

 #### Precision
- The Precision of the SVM model with Kernel as Linear is 0.826087
- The Precision of the SVM model with Kernel as rbf is 0.956522	
- The Precision of the SVM model with Kernel as poly is 0.970588
 
 #### Recall
- The Recall of the SVM model with Kernel as Linear is 0.863636
- The Recall of the SVM model with Kernel as rbf is 1.00
- The Recall of the SVM model with Kernel as poly is 1.00
 
 #### F1 Score
- The F1 Score of the SVM model with Kernel as Linear is 0.844444	
- The F1 Score of the SVM model with Kernel as rbf is 0.977778
- The F1 Score of the SVM model with Kernel as poly is 0.993976
 
 #### F2 Score
- The F2 Score of the SVM model with Kernel as Linear is 0.855856	
- The F2 Score of the SVM model with Kernel as rbf is 0.990991
- The F2 Score of the SVM model with Kernel as poly is 	0.993976


From the Performance dataframe we can observe that the C value is high for both the SVM kernel- rbf and the kernel- Poly
 where C is the regularization parameter that determines the width of the margin and the no of violations. 
 In our case C is valued as 100 for both Kernels RBF and Poly that means the margin will be small  and the no of violations will be less which will be sign of overfitting and we can see that the boost in the accuracy from 0.895 when the kernel is Linear  to 0.985(Kernel- rbf) and 0.990(Kernel-Poly) and the recall is also equivalent to 1 for these both kernels(Poly,rbf) which can also be a sign of overfitting.However we should also consider the fact that the dataset is quite small.The SVM Linear model is the best model for predicting the target Crop_yield as the C value is minimum which is 0.1 which means the more regularization has been performed and the margin will be quite wider hyperplane seperating the classses.The decision boundary is also smoother which may generalize and prevents over fitting of the model.When we look at the confusion matrix of the SVM linear model the True Positives are 122 which means the model have predicted that there will be crop yield when there is actually crop yield and the True negatives are also 57 which means there is no crop yield then the model have predicted it truly and classified as Neative.We can cosider that the SVM Linear Kernel is better model for predicting the Crop Yield.